In [1]:
import requests
from bs4 import BeautifulSoup as bs
import time

In [2]:
"""""url = "https://in.investing.com/equities/reliance-industries-historical-data"
page = requests.get(url)
 
print(page.status_code)"""""

'""url = "https://in.investing.com/equities/reliance-industries-historical-data"\npage = requests.get(url)\n \nprint(page.status_code)'

In [3]:
start_time = time.time()

url = "https://finance.yahoo.com/quote/RELIANCE.NS/history/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAAMev90NP1kg5rKG76nspf-sldvmyxpQLC2kMwhBM9WHCf3nTFo6i6F4IHn8wpeReMmZTFVEk_7aqKbIhJo7uiowqz1thLCed-4cpVqTLoBZuxubREHMguWY_aExKmQmruyC8k8qz7lUInHcRocSczrdsmn6mciLXV2sJnHeQvXf_&period1=1388534400&period2=1721399574"

values_list = []

webpage = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
print(webpage.status_code)

soup = bs(webpage.text, "html.parser")

stock_table = soup.find("table", class_="table yf-ewueuo")
tr_tag_list = stock_table.find_all("tr", class_="yf-ewueuo")

for each_tr_tag in tr_tag_list[1:]:
    td_tag_list = each_tr_tag.find_all("td", class_="yf-ewueuo")
    
    row_values = []
    for each_td_tag in td_tag_list[0:]:
        new_value = each_td_tag.text.strip()
        row_values.append(new_value)
    values_list.append(row_values)
    

print(values_list)
print("---- %s seconds ----" % (time.time() - start_time))

200
[['Jul 19, 2024', '3,172.00', '3,179.95', '3,105.60', '3,110.30', '3,110.30', '6,570,195'], ['Jul 18, 2024', '3,152.00', '3,187.80', '3,115.65', '3,173.35', '3,173.35', '6,149,187'], ['Jul 16, 2024', '3,187.00', '3,200.00', '3,138.25', '3,152.50', '3,152.50', '4,693,419'], ['Jul 15, 2024', '3,208.90', '3,211.70', '3,171.00', '3,194.45', '3,194.45', '2,664,844'], ['Jul 12, 2024', '3,169.00', '3,210.30', '3,149.00', '3,193.45', '3,193.45', '6,462,392'], ['Jul 11, 2024', '3,175.00', '3,208.50', '3,141.00', '3,161.30', '3,161.30', '5,970,473'], ['Jul 10, 2024', '3,190.00', '3,195.00', '3,126.30', '3,168.45', '3,168.45', '3,567,289'], ['Jul 9, 2024', '3,195.20', '3,201.00', '3,161.00', '3,180.55', '3,180.55', '3,480,001'], ['Jul 8, 2024', '3,178.00', '3,217.60', '3,165.05', '3,201.80', '3,201.80', '4,750,403'], ['Jul 5, 2024', '3,107.65', '3,197.00', '3,096.00', '3,177.25', '3,177.25', '6,134,855'], ['Jul 4, 2024', '3,114.00', '3,135.00', '3,101.60', '3,108.05', '3,108.05', '3,418,588']

In [4]:
start_time = time.time()

def extract_header_text(th_tag):
    # Extract all text nodes and join them
    texts = [text for text in th_tag.contents if isinstance(text, str)]
    main_text = ''.join(texts).strip()
    return main_text

headers_list = []

webpage = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
print(webpage.status_code)

soup = bs(webpage.text, "html.parser")

stock_table = soup.find("table", class_="table yf-ewueuo")
tr_tag_list = stock_table.find_all("tr", class_="yf-ewueuo")

for each_tr_tag in tr_tag_list[0:1]:
    th_tag_list = each_tr_tag.find_all("th", class_="yf-ewueuo")
    
    row_headers = []
    for each_th_tag in th_tag_list[0:]:
        new_value = extract_header_text(each_th_tag)
        row_headers.append(new_value)
    headers_list.append(row_headers)

# Flatten the headers_list if it is a list of lists
headers_list = headers_list[0] if headers_list else []

print(headers_list)
print("---- %s seconds ----" % (time.time() - start_time))

200
['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
---- 5.170093059539795 seconds ----


In [5]:
import pandas as pd
import csv
from datetime import datetime

In [6]:
def format_date(date_str):
    # Convert date from 'Jul 19, 2024' to '19-07-2024'
    date_obj = datetime.strptime(date_str, '%b %d, %Y')
    return date_obj.strftime('%d-%m-%Y')

def is_valid_row(row):
    # Check if the row is valid by ensuring the first column (Date) and some other expected columns are present
    if len(row) < 7:
        return False
    if 'Dividend' in row[1]:  # Example condition to filter out rows containing "Dividend"
        return False
    return True

# Apply date formatting to each row and filter out invalid rows
formatted_values_list = []
for row in values_list:
    if is_valid_row(row):
        formatted_row = [format_date(row[0])] + row[1:]
        formatted_values_list.append(formatted_row)

# Reverse the order of the rows
formatted_values_list.reverse()

# File name
filename = "Reliance_Industries_stock_data.csv"

# Writing to CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the headers
    writer.writerow(headers_list)
    
    # Write the data rows in reversed order
    writer.writerows(formatted_values_list)

print(f"Data has been written to {filename}")

Data has been written to Reliance_Industries_stock_data.csv
